## Pre-testing the implimenation of the magnetic probes.

Figuring out things before putting into the probe class.

In [3]:
# Create the machine, which specifies coil locations
# and equilibrium, specifying the domain to solve over
# this has to be either
# freegs.machine.MASTU(), in which case:
#tokamak = freegs.machine.MASTU()
# or
# MASTU_coils.MASTU_wpass()

%load_ext autoreload
%autoreload 2 


import os
import numpy as np
os.environ["ACTIVE_COILS_PATH"] = "../machine_configs/MAST-U/active_coils.pickle"
os.environ["PASSIVE_COILS_PATH"] = "../machine_configs/MAST-U/passive_coils.pickle"
os.environ["WALL_PATH"] = "../machine_configs/MAST-U/wall.pickle"
os.environ["LIMITER_PATH"] = "../machine_configs/MAST-U/limiter.pickle"
os.environ["PROBE_PATH"] = "../machine_configs/MAST-U/magnetic_probes.pickle"

from freegsnke import machine_config
from freegsnke import build_machine
from freegsnke import faster_shape
from freegsnke import magnetic_probes 
import freegs
import pickle 

tokamak = build_machine.tokamak()


# Creates equilibrium object 
eq = freegs.Equilibrium(tokamak=tokamak,
                        #domains can be changed 
                        Rmin=0.1, Rmax=2.0,    # Radial domain
                        Zmin=-2.2, Zmax=2.2,   # Height range
                        #grid resolution can be changed
                        nx=65, ny=129, # Number of grid points
                        # psi=plasma_psi[::2,:])   
                        )  



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

current_list = eq.tokamak.getCurrents()
print(current_list)

coil_names = [name for name in current_list]
print(coil_names)

In [ ]:
coil_names = [el for i, el in  enumerate(eq.tokamak.getCurrents())]
print(coil_names)

In [ ]:
probe_path = os.environ.get("PROBE_PATH",None)

with open(probe_path, 'rb') as file:
    probe_dict = pickle.load(file) 

floops = probe_dict['flux_loops']
pickups = probe_dict['pickups']

floops_small = floops[:]
floops_pos = np.array([item['position'] for item in floops_small])
floops_pos_R = np.array([item['position'][0] for item in floops_small])
floops_pos_Z = np.array([item['position'][1] for item in floops_small])

print(floops_pos)
print(floops_pos_R)
print(floops_pos_Z)
print(floops_pos_R.shape)
# probes = magnetic_probes.probe(eq)

In [ ]:
print(floops)

In [ ]:
eq.tokamak.coils


In [ ]:
eq.tokamak.createPsiGreens(floops_pos_R[0],floops_pos_Z[0])
current_list = eq.tokamak.getCurrents()

greens_2darray_floops = []
for item in floops_small:
    greens_2darray_floops.append(eq.tokamak.createPsiGreens(item['position'][0],item['position'][0]))

In [ ]:
# print(len(current_list))
# print(len(greens_2darray_floops[0]))
# greens_list = []
# psi_list = []
# for item in current_list:
#     greens_list.append(greens_2darray_floops[0][item])

# for item in greens_list:
#     psi_list.append()

# print(len(current_list))
# print(len(greens_list))



In [ ]:
greens_2darray_floops
print(len(greens_2darray_floops))
arr =  np.array([1,2,3,4,5])
newarr = arr[:,np.newaxis]

mult = []
for el in greens_2darray_floops:
    # mult.append(greens_2darray_floops[el]*5)
    print(el)



In [ ]:
print(pickups)
pickup_orientation = [el['orientation_vector'] for el in pickups]
pickup_position = [el['position'] for el in pickups]
print(pickup_orientation)

pickup_orientation[1][0]

In [ ]:

def pickup_value(eq):
        """ 
        Computes value of B.n at each pickup probe postition.
        Uses Br,Btor,Bz methods defined already for equilibrium object. 
        # i'm asumming that these already do the appropriate sums over the coils/grid/plasma etc.
        """
        BdotN = []
        for i, el in enumerate(pickups):
            # only need r, z positions as B = B(r,z)
            R,Z = pickup_position[i][0], pickup_position[i][2]
            orientation_vec = pickup_orientation[i]
            Bvec = [eq.Br(R,Z),eq.Btor(R,Z),eq.Bz(R,Z)]

            BdotN.append(np.dot(Bvec,orientation_vec))
        
        return BdotN



In [ ]:
from freegsnke import machine_config
coildict = machine_config.coils_dict

from freegs.gradshafranov import Greens ,GreensBr, GreensBz

 

In [ ]:
print(coildict.keys())
coildict['D1']['coords'][0]
# coildict['D1']['coords'][0][:,np.newaxis]



In [ ]:
greens_array = greens_coils()
greens_array


In [ ]:
# Numpy things... multiplyting arrays - matrix mult, 
list_1 = np.array([1,2,3,4,5,6,7,8,9])
list_2 = np.array([5,4,3])
list_3 = np.array([6,7,3])
# multiplication is element wise
print(list_2*list_3)
print(list_2[:,np.newaxis])
print(list_2[np.newaxis,:])

print('list 1 = {}, \nlist 2 = {}'.format(list_1,list_2))
print('scalar multiplication \n',list_1 *3 )
print('outerproduct v1 \n',list_1 *list_2[:,np.newaxis])
print('outerproduct v2 \n',list_1[:,np.newaxis]* list_2)
print('outerproduct v3 \n',list_1[np.newaxis,:]*list_2[:,np.newaxis])
print('sum v1 axis 0 \n', np.sum(list_1 *list_2[:,np.newaxis],axis = 0))
print('sum v1 axis 1\n ', np.sum(list_1 *list_2[:,np.newaxis],axis = 1))
print('sum v2 axis = 0\n', np.sum(list_1[:,np.newaxis]* list_2,axis = 0))
print('sum v2 axis = 1\n', np.sum(list_1[:,np.newaxis]* list_2,axis = 1))

In [ ]:
# joining numpy arrays - np version of append...
vstack = np.vstack((list_2,list_2))
hstack = np.hstack((list_2,list_3))
hstack2 = np.hstack((list_1,list_2))
concat = np.concatenate((list_1,list_2),axis= 0)
print(vstack)
print(hstack)
print(hstack2)
print(concat)

In [ ]:
print(Greens(1,1,list_3,list_3))
list_4 = []
for i , el in enumerate(list_1):
    list_4.append(Greens(el,el,list_2[:,np.newaxis],list_2[:,np.newaxis]))
print(list_4)
print(np.sum(list_4, axis = 0))


In [ ]:
def fun(x,y):
    return x + y

In [ ]:
y1 = fun(list_1,4)
y2 = fun(list_1,list_2[:,np.newaxis])
print(y2)
y2.shape
y3 = Greens(list_1,list_1,list_2[:,np.newaxis],list_3[:,np.newaxis])
print(y3)
print(y3.shape)



In [ ]:
div = 1 / list_1
div 

In [ ]:
# print(coildict[key]['coords'])
# print(floops_pos_R[:,np.newaxis])

def greens_coils():
    greens_array = []
    for key in coildict.keys():
            pol = coildict[key]['polarity'][:,np.newaxis]
            mul = coildict[key]['multiplier'][:,np.newaxis]
            greens_array.append(np.sum(Greens(coildict[key]['coords'][0],
                                   coildict[key]['coords'][1],
                                   floops_pos_R[:,np.newaxis],
                                   floops_pos_Z[:,np.newaxis]),axis= 1))
            # greens_array.append(Greens(coildict[key]['coords'][0][:,np.newaxis],
            #                        coildict[key]['coords'][1][:,np.newaxis],
            #                        floops_pos_R,
            #                        floops_pos_Z))
    return greens_array
def greens_coils2():
    greens_array = []
    for key in coildict.keys():
            greens_array.append(Greens(coildict[key]['coords'][0],
                                   coildict[key]['coords'][1],
                                   floops_pos_R[:,np.newaxis],
                                   floops_pos_Z[:,np.newaxis]))
            
    return greens_array

def greens_coils3(key):
    pol = coildict[key]['polarity'][np.newaxis,:]
    mul = coildict[key]['multiplier'][np.newaxis,:]
    greens = Greens(coildict[key]['coords'][0],
                                   coildict[key]['coords'][1],
                                   floops_pos_R[:,np.newaxis],
                                   floops_pos_Z[:,np.newaxis])
    greens *= pol
    greens *= mul
    return greens
def greens_4():
    arr = []
    for key in coildict.keys():
            a = np.sum(greens_coils3(key),axis =1)
            arr.append(a)
    return np.array(arr) 

In [ ]:
a = greens_coils3('Solenoid')
print(a)
print(a.shape)
print(np.sum(a,axis=1))
b = greens_4()
print(b)
print(b.shape)
c = np.sum(b,axis=0)
print(c)


In [ ]:
pol = coildict['Solenoid']['polarity'][np.newaxis,:]
mul = coildict['Solenoid']['multiplier'][:,np.newaxis]

coiltest = greens_coils3('Solenoid')
print(coiltest.shape)
print(len(pol))
print(pol.shape)
coiltestnew = coiltest*pol
print(coiltest)
print(coiltestnew.shape)


In [ ]:
print(len(coildict['D1']['coords'][0]))
print(len(coildict['D1']['multiplier']))



In [ ]:
array = np.array([[1,2,3],[4,5,6]])
array2  =np.array([1,-1,0])
array*array2

In [ ]:
coil_names

array_of_coil_currents = np.array([eq.tokamak[label].current for label in current_list])
greens_all_coils = greens_4()
print(len(array_of_coil_currents))
print(len(greens_all_coils))

psi_all_coils = greens_all_coils * array_of_coil_currents[:,np.newaxis]

psi_sum_all_coils = np.sum(psi_all_coils,axis = 0)
print(psi_sum_all_coils)

In [ ]:
nx ,ny = 11,6
Rmin, Rmax = 0,10
Zmin, Zmax = 5 ,10

R_1D = np.linspace(Rmin, Rmax, nx)
Z_1D = np.linspace(Zmin, Zmax, ny)
R, Z = np.meshgrid(R_1D, Z_1D, indexing="ij")

In [ ]:
R[np.newaxis , :,:]

In [ ]:
X = Greens(R[:,:,np.newaxis],Z[:,:,np.newaxis],floops_pos_R[np.newaxis,np.newaxis,:],floops_pos_Z[np.newaxis,np.newaxis,:])
Y = np.sum(X,axis = (1,2))
print(X.shape)
print(Y.shape)

In [ ]:
eq._profiles.jtor

In [ ]:
abc = [[1,2],[3,4],[5,6],[7,8]]
abc[0][0]

## Interfacing work 